## Exercice

Faire un programme qui:

- Trouve les 10 villes de France les + peuplées (e.g https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées)
- Pour chaque ville, trouve sa distance avec les autres.
    - par exemple, avec google maps api (nécessite de s'authentifier / créer un token) avec https://github.com/googlemaps/google-maps-services-python (cf. doc token dans le readme)
    - ou avec https://fr.distance24.org/ (pas besoin de s'authentifier)
- Trouve les villes les plus proches

In [2]:
import asyncio
from pyppeteer import launch # conda install -c conda-forge pyppeteer
from functools import reduce
import requests
import pandas as pd
import itertools

In [3]:
# //*[@id="mw-content-text"]/div/table[1]/tbody/tr[1]/td[2]
browser = await launch()
page = await browser.newPage()
await page.goto('https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées')
#citiesTd = await page.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[2]')
citiesTr = await page.J("#mw-content-text div table tbody tr") #querySelectorAll
cities  = await page.JJeval("td:nth-child(2) a", '(nodes => nodes.map(n => n.innerText))', citiesTr)# evaluate

In [4]:
cities = cities[:10]
cities

['Paris',
 'Marseille',
 'Lyon',
 'Toulouse',
 'Nice',
 'Nantes',
 'Montpellier',
 'Strasbourg',
 'Bordeaux',
 'Lille']

In [5]:
#gmaps = googlemaps.Client(client_id=, client_secret=client_secret)
def getDistance(a, b):
    json = requests.get("https://fr.distance24.org/route.json?stops=%s|%s" % (a, b)).json()
    return json['distance']
    
getDistance('Paris', 'Marseille')

661

In [8]:
combinations = list(itertools.combinations(cities,2))
distances = map(lambda a: (a[0], a[1], getDistance(a[0], a[1])), combinations)
df = pd.DataFrame(distances, columns=('De', 'A', 'km'))

In [9]:
df3 = df.set_index(['De', 'A']).sort_index()

In [10]:
df3

km
De          A               
Bordeaux    Lille        700
Lyon        Bordeaux     436
            Lille        558
            Montpellier  251
            Nantes       516
            Nice         300
            Strasbourg   384
            Toulouse     360
Marseille   Bordeaux     506
            Lille        835
            Lyon         278
            Montpellier  126
            Nantes       696
            Nice         159
            Strasbourg   617
            Toulouse     319
Montpellier Bordeaux     381
            Lille        784
            Strasbourg   629
Nantes      Bordeaux     276
            Lille        508
            Montpellier  584
            Strasbourg   710
Nice        Bordeaux     638
            Lille        835
            Montpellier  273
            Nantes       792
            Strasbourg   545
Paris       Bordeaux     500
            Lille        204
            Lyon         392
            Marseille    661
            Montpellier  596
            Nantes       343
            Nice         688
            Strasbourg   397
            Toulouse     589
Strasbourg  Bordeaux     760
            Lille        408
Toulouse    Bordeaux     212
            Lille        792
            Montpellier  196
            Nantes       466
            Nice         469
            Strasbourg   737

In [30]:
products = list(itertools.product(cities, repeat=2))
distances2 = map(lambda a: (a[0], a[1], getDistance(a[0], a[1])), products)
df2 = pd.DataFrame(distances2, columns= ('De', 'A', 'km'))

In [31]:
df3 = df2.set_index(['De', 'A']).sort_index()
df3.unstack()

km                                                     \
A           Bordeaux Lille Lyon Marseille Montpellier Nantes Nice Paris   
De                                                                        
Bordeaux           0   700  436       506         381    276  638   500   
Lille            700     0  558       835         784    508  835   204   
Lyon             436   558    0       278         251    516  300   392   
Marseille        506   835  278         0         126    696  159   661   
Montpellier      381   784  251       126           0    584  273   596   
Nantes           276   508  516       696         584      0  792   343   
Nice             638   835  300       159         273    792    0   688   
Paris            500   204  392       661         596    343  688     0   
Strasbourg       760   408  384       617         629    710  545   397   
Toulouse         212   792  360       319         196    466  469   589   

                                 
A           Strasbourg Toulouse  
De                               
Bordeaux           760      212  
Lille              408      792  
Lyon               384      360  
Marseille          617      319  
Montpellier        629      196  
Nantes             710      466  
Nice               545      469  
Paris              397      589  
Strasbourg           0      737  
Toulouse           737        0